## Memory Utilization (the computer kind)

While scaling up code for big data analyses, performance and scaling requirements make it important to gain the skills to predict RAM requirements and monitor RAM consumption in order to maximally utilize resources.  This also helps to identify areas of analyses that need modification in order to run within the physical constraints of a particular system.

In [17]:
import numpy as np
import psutil
import sys
import os
import gc

In [2]:
big_array = np.zeros((1024, 1024, 128))

In [3]:
# Dimensions
big_array.shape

(1024, 1024, 128)

In [4]:
# Data being stored
big_array.dtype

dtype('float64')

In [5]:
# Storage per element
big_array.dtype.itemsize

8

In [6]:
# Total element count: N_x * N_y * N_z
np.prod(big_array.shape)

134217728

In [7]:
# Total storage
np.prod(big_array.shape) * big_array.dtype.itemsize

1073741824

In [8]:
# Human readable:
def PrintArraySize(a):
  count = np.prod(a.shape)
  elem_size = a.dtype.itemsize
  print(f'{count*elem_size/1024**3} GB')

PrintArraySize(big_array)

1.0 GB


In [9]:
psutil.virtual_memory()

svmem(total=134775570432, available=112133500928, percent=16.8, used=18995654656, free=16937414656, active=73788833792, inactive=40268271616, buffers=0, cached=98842501120, shared=3005235200, slab=2562609152)

In [10]:
psutil.Process(os.getpid())

psutil.Process(pid=99261, name='python', status='running', started='12:41:32')

In [11]:
# rss is "Resident Set Size",
# vms is "Virtual Memory Size" which includes other aspects of memory such
# as memory swapped to disk (not in use on Rhino) and that might be
# copy-on-write shared, such as shared library memory that typically won't
# count against usage limits.
# These values right here are low because of Linux "over-commit" features.
# It will only assign real memory once it's used, not when it's reserved.
psutil.Process(os.getpid()).memory_info()

pmem(rss=76824576, vms=2089955328, shared=14094336, text=3342336, lib=0, data=1900896256, dirty=0)

Checking your active processes. ssh into rhino (or open a terminal), and run:
```squeue -u your_username```

Your JupyterLab server will look like "spawner-", and will have a node associated.  Your cluster jobs will have names based on the functions you launched.  You can connect to the node for an active job with ssh like:
```ssh node42```

Then you can run the program "top" where shift-P sorts processes by processor load, and shift-M sorts processes by memory load.  Sometimes this is helpful for quickly seeing the load of a running process.

In [12]:
# Now let's actually use the memory!  We'll assign 1 to every entry.
big_array[:] = 1

In [13]:
# Suddenly Linux recognizes that this is real memory in real use,
# and gives it physical storage!
psutil.Process(os.getpid()).memory_info()

pmem(rss=1151201280, vms=2089955328, shared=14270464, text=3342336, lib=0, data=1900896256, dirty=0)

In [14]:
psutil.Process(os.getpid()).memory_info().rss / 1024**3

1.0721473693847656

In [15]:
# Clean-up usage.  Note, this only sometimes works right away!
big_array = None
psutil.Process(os.getpid()).memory_info().rss / 1024**3

0.07214736938476562

In [18]:
big_aray = None
# If you need a stronger guarantee of clean-up (rarely needed),
# you can try calling the garbage collector manually
gc.collect()
psutil.Process(os.getpid()).memory_info().rss / 1024**3

0.07846832275390625

In [20]:
def WorkFunction():
  big_array = np.zeros((1024, 1024, 128))
  big_array[:] = 1
  print(psutil.Process(os.getpid()).memory_info().rss / 1024**3)

WorkFunction()
# Variables are cleaned up when the function returns!
print(psutil.Process(os.getpid()).memory_info().rss / 1024**3)

1.0783843994140625
0.07860946655273438


In [21]:
# It's easy to build up memory utilization as you keep making modified copies.
big_array = np.zeros((1024, 1024, 128))
big_array[:] = 1
big_array2 = 2*big_array
print(psutil.Process(os.getpid()).memory_info().rss / 1024**3)

2.0785980224609375


In [23]:
# This manually removes variables, cleans up like assigning to None except
# the variable has no definition anymore.
del big_array
del big_array2

In [24]:
# Numpy arrays are more performant modifying in place.
big_array = np.zeros((1024, 1024, 128))
big_array[:] = 1
big_array *= 2
print(psutil.Process(os.getpid()).memory_info().rss / 1024**3)

1.0786323547363281


In [25]:
# Dimensionally reduced data is miniscule:
big_array = np.zeros((1024, 1024, 128))
big_array[:] = 1
big_array_mean = np.mean(big_array, axis=0)
print(psutil.Process(os.getpid()).memory_info().rss / 1024**3)

1.079498291015625


In [26]:
del big_array

In [28]:
# Smart approach!
def MakeReducedData():
  big_array = np.zeros((1024, 1024, 128))
  big_array[:] = 1
  big_array_mean = np.mean(big_array, axis=0)
  print(psutil.Process(os.getpid()).memory_info().rss / 1024**3)
  return big_array_mean

big_array_mean = MakeReducedData()
print(psutil.Process(os.getpid()).memory_info().rss / 1024**3)

1.0805892944335938
0.08058547973632812


## File Input/Output (IO)

Continuing on in Big Data analyses will give you an increased need to think carefully about file IO, as you migrate from using premade data to being a generator of data.

### Temporary / Intermediate data

This kind of data is transient, used in the middle of a calculation for storing a result for minutes, hours, or maybe up to a couple months.  It is data that typically does not need to be backed up, and where you are not worried about whether or not it can be accessed years later, or work on new environments or new computers or even on anyone else's computer or language.  You just need it to "work" cleanly and simply for short term work, usually to avoid recalculating things.

Good examples of this in Python are pickle and numpy saved files.  Pickle is a "do not share it" data format, because there are security considerations where loading pickled data can actually execute code in the file.  But since there is little chance of you hacking yourself, you can gain the advantages of quickly streaming to disk arbitrary Python objects and reloading them.  Pickled data is NOT compatible if, for example, you save data for an object of a class type, and then change the class!  The same type needs to be available when you reload.  This means it will often stop working even on standard types or common library types if you upgrade to a new version of Python or new environment.  Hence, it is a powerful and useful temporary local-use format.

In [30]:
import pickle
d = {'Bob': 34, 'Alice': 43, 'Joe': 25, 'Susan': 36}
with open('my_dictionary.pkl', 'wb') as fw:
  pickle.dump(d, fw)

In [31]:
with open('my_dictionary.pkl', 'rb') as fr:
  d2 = pickle.load(fr)


{'Bob': 34, 'Alice': 43, 'Joe': 25, 'Susan': 36}


Saving with numpy, such as np.save or np.savez is similar, as some things you might save with numpy are actually pickled to do it!  Pure numerical data saved with numpy has slightly longer longevity, and non-pickled data saved with numpy can be shared, but it is probably not a format to trust for very long term storage.

In [34]:
a = np.ones((1024, 32))
print(np.prod(a.shape)*a.dtype.itemsize)
np.save('my_array.npy', a)
os.path.getsize('my_array.npy')

262144


262272

Notice numpy stores binary data compactly!  This is an asset for large data.

### Long term archival of data

If your data is small dimensionally reduced data, however, other formats like csv (comma-separated values) can be very convenient for long term archival (decades through human lifetime).  This matters a lot for valuable data!

In [37]:
import pandas as pd
pd.DataFrame(a).to_csv('my_array.csv')
os.path.getsize('my_array.csv')

135169

Here csv looks smaller!  But that's deceptive, because we're actually storing "1, 1, 1, 1, 1, ..." which is very compact because we used all ones.

In [38]:
a = np.random.random((1024, 32))
print(np.prod(a.shape)*a.dtype.itemsize)
np.save('my_array2.npy', a)
os.path.getsize('my_array2.npy')

262144


262272

In [39]:
pd.DataFrame(a).to_csv('my_array2.csv')
os.path.getsize('my_array2.csv')

635236

In [40]:
# Let's look at the first 5 lines of the file the manual way:
with open('my_array2.csv', 'r') as fr:
  lines = fr.readlines()
  print('\n'.join(lines[0:5]))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31

0,0.739889406208754,0.2827046585389744,0.5594041053106992,0.5633814456955742,0.2552310893292975,0.622916572914727,0.7062238681696796,0.6854959592756991,0.05114680974080621,0.5794636349889507,0.39137033965991064,0.7190044458551405,0.8577175896018705,0.17390413774348434,0.9599656980161152,0.16408806183128533,0.6129906461368634,0.1570640515473538,0.4618601748807176,0.7720280759890569,0.23839910442168866,0.3550156568848738,0.8167389221351987,0.6049325039872016,0.7643235202997138,0.8579450885132974,0.18124433594727918,0.9737158705078566,0.025460105758991824,0.8850167581631706,0.6784267570443804,0.45367117021300396

1,0.2918858553351016,0.9355939157922352,0.1945736169588289,0.6453342152398456,0.3661013387218227,0.21553416570966366,0.9736155533078619,0.5522047771507568,0.20369459218080765,0.9814204594871014,0.3867461700787206,0.9699237534278549,0.8320965570814466,0.6261214497253011,0.3247173015409378,0.137

In [44]:
# Maybe we just wanted the data, and not the headers and indices...
pd.DataFrame(a).to_csv('my_array3.csv', header=None, index=None)
os.path.getsize('my_array3.csv')

631139

In [45]:
# Now let's look at the first 5 lines of the file the manual way:
with open('my_array3.csv', 'r') as fr:
  lines = fr.readlines()
  print('\n'.join(lines[0:5]))

0.739889406208754,0.2827046585389744,0.5594041053106992,0.5633814456955742,0.2552310893292975,0.622916572914727,0.7062238681696796,0.6854959592756991,0.05114680974080621,0.5794636349889507,0.39137033965991064,0.7190044458551405,0.8577175896018705,0.17390413774348434,0.9599656980161152,0.16408806183128533,0.6129906461368634,0.1570640515473538,0.4618601748807176,0.7720280759890569,0.23839910442168866,0.3550156568848738,0.8167389221351987,0.6049325039872016,0.7643235202997138,0.8579450885132974,0.18124433594727918,0.9737158705078566,0.025460105758991824,0.8850167581631706,0.6784267570443804,0.45367117021300396

0.2918858553351016,0.9355939157922352,0.1945736169588289,0.6453342152398456,0.3661013387218227,0.21553416570966366,0.9736155533078619,0.5522047771507568,0.20369459218080765,0.9814204594871014,0.3867461700787206,0.9699237534278549,0.8320965570814466,0.6261214497253011,0.3247173015409378,0.1373044136552175,0.33884331114953936,0.5379205490815059,0.318108531621233,0.8636968194106976,0.

The json format is another text format for data which retains human-readable properties, and can be expected to have long term archival properties.  As a text format it is easy to work with but often not compact for numerical data, so it is an appropriate store for dimensionally reduced results and smaller data that you want to preserve for a long time.

In [48]:
import json
d = {'Bob': 34, 'Alice': 43, 'Joe': 25, 'Susan': 36, 'TheSmithBrothers': [29, 31]}
with open('the_ages.json', 'w') as fw:
  json.dump(d, fw)
os.path.getsize('the_ages.json')

78

In [49]:
with open('the_ages.json', 'r') as fr:
  d2 = json.load(fr)
print(d2)

{'Bob': 34, 'Alice': 43, 'Joe': 25, 'Susan': 36, 'TheSmithBrothers': [29, 31]}


In [50]:
# Let's see what we stored.
with open('the_ages.json', 'r') as fr:
  lines = fr.readlines()
  print('\n'.join(lines))

{"Bob": 34, "Alice": 43, "Joe": 25, "Susan": 36, "TheSmithBrothers": [29, 31]}


The json format stores as human readable text, and in fact looks a lot like Python code!  This is a very future-save archival format for data that fits into the supportd types.

For long term compact archiving of large binary data, you will want to do some reading for the particular type of data you are trying to store, and choose a format that you estimate will work well for this and retain long support.  There is no one answer for this.